In [1]:
import duckdb
import pandas as pd

con = duckdb.connect("../data_processed/nyc_taxis.duckdb")


In [2]:
con.execute("SHOW TABLES").fetchall()


[('trips_all',),
 ('trips_clean',),
 ('trips_feb',),
 ('trips_jan',),
 ('trips_mar',)]

In [3]:
df = con.execute("""
SELECT *
FROM trips_clean
LIMIT 10000
""").fetchdf()

df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duree_min,heure,jour_semaine,mois
0,2,2016-01-01,2016-01-01,2,1.10,-73.990372,40.734695,1,N,-73.981842,...,0.5,0.5,0.0,0.0,0.3,8.8,0,0,5,1
1,2,2016-01-01,2016-01-01,5,4.90,-73.980782,40.729912,1,N,-73.944473,...,0.5,0.5,0.0,0.0,0.3,19.3,0,0,5,1
2,2,2016-01-01,2016-01-01,1,10.54,-73.984550,40.679565,1,N,-73.950272,...,0.5,0.5,0.0,0.0,0.3,34.3,0,0,5,1
3,2,2016-01-01,2016-01-01,1,4.75,-73.993469,40.718990,1,N,-73.962242,...,0.0,0.5,0.0,0.0,0.3,17.3,0,0,5,1
4,2,2016-01-01,2016-01-01,3,1.76,-73.960625,40.781330,1,N,-73.977264,...,0.0,0.5,0.0,0.0,0.3,8.8,0,0,5,1


In [4]:
con.execute("""
SELECT COUNT(*) AS nb_lignes
FROM trips_clean
""").fetchdf()


,nb_lignes
0,34486592


In [5]:
con.execute("""
SELECT 
  MIN(tpep_pickup_datetime) AS debut,
  MAX(tpep_pickup_datetime) AS fin
FROM trips_clean
""").fetchdf()


,debut,fin
0,2016-01-01,2016-03-31 23:59:59


In [6]:
# Durée
con.execute("""
SELECT 
  MIN(duree_min) AS duree_min,
  MAX(duree_min) AS duree_max,
  AVG(duree_min) AS duree_moy
FROM trips_clean
""").fetchdf()


,duree_min,duree_max,duree_moy
0,0,192313,15.557718


In [7]:
# Distance
con.execute("""
SELECT 
  MIN(trip_distance) AS dist_min,
  MAX(trip_distance) AS dist_max,
  AVG(trip_distance) AS dist_moy
FROM trips_clean
""").fetchdf()


,dist_min,dist_max,dist_moy
0,0.0,19072628.8,5.409641


In [8]:
# Montants
con.execute("""
SELECT 
  MIN(total_amount) AS total_min,
  MAX(total_amount) AS total_max,
  AVG(total_amount) AS total_moy
FROM trips_clean
""").fetchdf()


,total_min,total_max,total_moy
0,0.0,429562.25,15.78203


In [9]:
# Valeurs Null
con.execute("""
SELECT
  SUM(CASE WHEN passenger_count IS NULL THEN 1 ELSE 0 END) AS null_passenger_count,
  SUM(CASE WHEN trip_distance IS NULL THEN 1 ELSE 0 END) AS null_trip_distance,
  SUM(CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END) AS null_total_amount
FROM trips_clean
""").fetchdf()


,null_passenger_count,null_trip_distance,null_total_amount
0,0.0,0.0,0.0


In [10]:
# Distribution de paiment
con.execute("""
SELECT payment_type, COUNT(*) AS n
FROM trips_clean
GROUP BY payment_type
ORDER BY n DESC
""").fetchdf()


,payment_type,n
0,1,22989050
1,2,11339865
2,3,118614
3,4,39062
4,5,1


In [11]:
# Distribution passagers
con.execute("""
SELECT passenger_count, COUNT(*) AS n
FROM trips_clean
GROUP BY passenger_count
ORDER BY passenger_count
""").fetchdf()


,passenger_count,n
0,0,1631
1,1,24535605
2,2,4902727
3,3,1383147
4,4,657780
5,5,1852861
6,6,1152646
7,7,71
8,8,68
9,9,56


In [12]:
# Demande par heure
con.execute("""
SELECT heure, COUNT(*) AS nb_courses
FROM trips_clean
GROUP BY heure
ORDER BY heure
""").fetchdf()


,heure,nb_courses
0,0,1220879
1,1,890665
2,2,649844
3,3,490190
4,4,370226
5,5,349589
6,6,765571
7,7,1300413
8,8,1592651
9,9,1593332


In [13]:
# Demande par jour
con.execute("""
SELECT jour_semaine, COUNT(*) AS nb_courses
FROM trips_clean
GROUP BY jour_semaine
ORDER BY jour_semaine
""").fetchdf()


,jour_semaine,nb_courses
0,0,4554391
1,1,4431659
2,2,4786707
3,3,4992686
4,4,5228150
5,5,5368778
6,6,5124221


In [14]:
# Relation distance → montant
con.execute("""
SELECT 
  ROUND(trip_distance) AS dist_arr,
  AVG(total_amount) AS total_moy
FROM trips_clean
GROUP BY dist_arr
ORDER BY dist_arr
LIMIT 30
""").fetchdf()


,dist_arr,total_moy
0,0.0,9.009453
1,1.0,8.687948
2,2.0,12.584572
3,3.0,16.314563
4,4.0,19.712906
5,5.0,22.850427
6,6.0,25.849907
7,7.0,28.882024
8,8.0,32.858449
9,9.0,37.790492


In [ ]:
# KPI global
con.execute("""
COPY (
    SELECT 
        COUNT(*) AS nb_courses,
        SUM(total_amount) AS revenu_total,
        AVG(total_amount) AS ticket_moyen,
        AVG(trip_distance) AS distance_moyenne,
        AVG(duree_min) AS duree_moyenne
    FROM trips_clean
)
TO '../data_processed/kpi_global.csv'
WITH (HEADER, DELIMITER ',');
""")


In [ ]:
# KPI par heure
con.execute("""
COPY (
    SELECT 
        heure,
        COUNT(*) AS nb_courses,
        SUM(total_amount) AS revenu_total,
        AVG(total_amount) AS ticket_moyen
    FROM trips_clean
    GROUP BY heure
    ORDER BY heure
)
TO '../data_processed/kpi_par_heure.csv'
WITH (HEADER, DELIMITER ',');
""")


In [ ]:
# KPI jour x Heure (heatmap)
con.execute("""
COPY (
    SELECT 
        jour_semaine,
        heure,
        COUNT(*) AS nb_courses,
        SUM(total_amount) AS revenu_total
    FROM trips_clean
    GROUP BY jour_semaine, heure
    ORDER BY jour_semaine, heure
)
TO '../data_processed/kpi_jour_heure.csv'
WITH (HEADER, DELIMITER ',');
""")


In [ ]:
# KPI Géographique
con.execute("""
COPY (
    SELECT 
        ROUND(pickup_latitude, 2) AS lat,
        ROUND(pickup_longitude, 2) AS lon,
        COUNT(*) AS nb_courses,
        SUM(total_amount) AS revenu_total
    FROM trips_clean
    GROUP BY lat, lon
    HAVING COUNT(*) > 100
    ORDER BY nb_courses DESC
)
TO '../data_processed/kpi_geo.csv'
WITH (HEADER, DELIMITER ',');
""")
